In [1]:
import pandas as pd

# Path to my file – it can be any file
file_path = "C:/Users/DELL/Downloads/deliveries (2).csv"

# Load the CSV file with the correct separator
df = pd.read_csv(file_path, sep=";", encoding="utf-8", engine="python")

# Settings to view all columns and full cell content, since after downloading the CSV it looked strange and I needed to make sure everything was in order and in place
pd.set_option('display.max_columns', None)        # Show all columns
pd.set_option('display.max_colwidth', None)       # Show full content of each cell
pd.set_option('display.width', None)              # Adjust width
pd.set_option('display.expand_frame_repr', False) # Don't split columns into multiple lines

df.head()


,id,campaign_id,campaign_name,newsletter_id,newsletter_name,template_id,template_name,transactional_message_id,transactional_message_name,subject,action_id,parent_action_id,customer_id,recipient,created,created_RFC3339,failure_message,sent,sent_RFC3339,delivered,delivered_RFC3339,opened,opened_RFC3339,clicked,clicked_RFC3339,converted,converted_RFC3339,bounced,bounced_RFC3339,spammed,spammed_RFC3339,unsubscribed,unsubscribed_RFC3339,suppressed,suppressed_RFC3339,failed,failed_RFC3339,drafted,drafted_RFC3339,topic_unsubscribed,topic_unsubscribed_RFC3339,email
0,RPCtBwUAAYw2N2cGLMlm-0X9RlZ8ZQ==,NaN,NaN,NaN,NaN,242,Low data,4.0,Low data,Running out of data?,NaN,NaN,dfbe3f4a5782a97646a6ce9cc91b234a,reggidori@allin-partners.com,1701716649,4/12/2023 14:04,"No MX for allin-partners.com, lookup allin-partners.com on 172.17.0.1:53: no such host",1701716655,4/12/2023 14:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701716658,4/12/2023 14:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,reggidori@allin-partners.com
1,RPCtBwUAAYw2JZ5ZnHbPWa9Mgfeamw==,NaN,NaN,NaN,NaN,244,Video uploaded,6.0,Video uploaded,Video successfully uploaded!,NaN,NaN,bb4f7c2e1397449ff338881f23de389d,hamdi@videoblogg.com,1701715484,4/12/2023 13:44,5.1.1 The email account that you tried to reach does not exist. Please try\n5.1.1 double-checking the recipient's email address for typos or\n5.1.1 unnecessary spaces. Learn more at\n5.1.1 https://support.google.com/mail/?p=NoSuchUser n8-20020a0cfbc8000000b0067a1fc7914bsi8523477qvp.236 - gsmtp,1701715486,4/12/2023 13:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701715487,4/12/2023 13:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hamdi@videoblogg.com
2,dgTwrQcAAMfGCMbGCAGMNcXPsI8buTfq_C49eIw=,28.0,Signup onboarding v3,NaN,NaN,223,Share live stream features,NaN,NaN,Take your streams to the next level with Dacast,419.0,NaN,eac2145c-eb90-a425-2793-16d360ca8172,maheshmahi@gamil.con,1701709205,4/12/2023 12:00,"No MX for gamil.con, lookup gamil.con on 172.17.0.1:53: no such host",1701709209,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701709213,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,maheshmahi@gamil.con
3,dgTwrQcAALjJCLfJCAGMNcXPpfPDnufKi-jyyz0=,28.0,Signup onboarding v3,NaN,NaN,205,Share live stream features,NaN,NaN,Take your streams to the next level with Dacast,372.0,NaN,f9a98d62-5945-adff-2229-8359d98ac536,anshul@realimapct.tv,1701709205,4/12/2023 12:00,"No MX for realimapct.tv, lookup realimapct.tv on 172.17.0.1:53: no such host",1701709207,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701709209,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anshul@realimapct.tv
4,dgTwrQcAAJnJCJjJCAGMNcXPpe2eqZyz7iMBYxs=,28.0,Signup onboarding v3,NaN,NaN,222,Onboarding Stream,NaN,NaN,Start streaming with Dacast today,418.0,NaN,1a506fdc-c117-8b76-7a5a-57aab8571700,johncookj11@netzero.com,1701709205,4/12/2023 12:00,temporary delay due to high volume,1701709207,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701709216,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,johncookj11@netzero.com


In [ ]:
# Filter the unique non-null values from the 'failure_message' column to see all possible email issues
unique_failures_df = pd.DataFrame(df['failure_message'].dropna().unique(), columns=['failure_message'])

# Sort by message to make the information easier to navigate
unique_failures_df = unique_failures_df.sort_values(by='failure_message').reset_index(drop=True)

# Save the original total count, just for reference
original_count = len(unique_failures_df)

unique_failures_df.head()


,failure_message
0,"""JunkMail rejected - a227-71.mailgun.net [143.55.227.71]:44494 is in an RBL\non rbl.websitewelcome.com, see """
1,"1 Requested mail action aborted, mailbox not found"
2,"30 Sorry, your message to ava@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
3,"30 Sorry, your message to bilallasuru@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
4,"30 Sorry, your message to cathiegriggs278@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."


In [ ]:
# Source: https://senderscore.org/assess/bounce-lookup/email-error/5-X-X/
# JUST CHANGE THE NUMBERS FOR THE ERROR YOU WANT TO LOOK UP
# I checked this page to be sure about what I was eliminating

#Due to ambiguity i will say NON EXISTENT ≠ NON FUNCTIONAL

# Filter unique non-null values from the 'failure_message' column to see all possible email errors
unique_failures_df = pd.DataFrame(df['failure_message'].dropna().unique(), columns=['failure_message'])

# Sort messages for easier review
unique_failures_df = unique_failures_df.sort_values(by='failure_message').reset_index(drop=True)

# Store the original total count for later comparison
original_count = len(unique_failures_df)

# Keywords to identify clearly non-existent user messages

# Due to ambiguity, I will define NO FUNCTIONAL =/= NON-EXISTENT

# Keywords that strongly indicate the email does not exist.
# I know I could use more specific phrases like 
# "The email account that you tried to reach does not exist" to avoid generalization,
# but in the case of this CSV, it’s not necessary. 
# I also know some of them overlap, but better to be safe.
sure_keywords = [
    "does not exist",
    "5.1.1",
    "user doesn't exist",
    "no such host",
    "user unknown",
    "domain accepts no mail",
    "invalid recipient",
    "mailbox not found" 
]

# COMMENTS ON EACH PATTERN THAT MAY INDICATE NON-EXISTENT EMAILS:

# "does not exist" – Indicates that the user does not exist in the system or domain (also covers "mailbox does not exist!")
# "user doesn't exist" – Similar to the above, requires apostrophe; means the user was not found
# "no such host" – Means the domain/server cannot be found; implies the email doesn't exist in the network
# "user unknown" – The system does not recognize the user
# "domain accepts no mail" – The domain refuses all mail; destination is invalid
# "5.1.1" – Common SMTP error code indicating invalid or non-existent address
# "invalid recipient" – Clearly states the recipient is invalid
# "mailbox not found" – Mailbox does not exist in the system

# Keywords that *suggest* the email might not exist, though they are not 100% definitive
suspicious_keywords = [
    "550 5.0.1",              # Usually means the user doesn't exist
    "sender verify failed"    # Often triggered when verifying a non-existent recipient
]

# Combine both groups of keywords
non_existent_keywords = sure_keywords + suspicious_keywords

# Function to check if a failure message contains any of the keywords (case-insensitive)
def contains_non_existent(msg):
    return any(keyword in msg.lower() for keyword in non_existent_keywords)

# Create a DataFrame with messages that indicate a non-existent user
no_existent_df = unique_failures_df[unique_failures_df['failure_message'].apply(contains_non_existent)]

# Remove those messages from the original list
unique_failures_df = unique_failures_df[~unique_failures_df['failure_message'].isin(no_existent_df['failure_message'])]

# Print counts to verify that the filtering was performed correctly
print(f"Original total in unique_failures_df: {original_count}")
print(f"Rows moved to no_existent_df: {len(no_existent_df)}")
print(f"Remaining in unique_failures_df: {len(unique_failures_df)}")


Original total in unique_failures_df: 109
Rows moved to no_existent_df: 52
Remaining in unique_failures_df: 57


In [4]:
# At first, the array was complete, but as I kept investigating, I started adding values to the list,
# and it gradually became smaller over time
unique_failures_df.head()

,failure_message
0,"""JunkMail rejected - a227-71.mailgun.net [143.55.227.71]:44494 is in an RBL\non rbl.websitewelcome.com, see """
2,"30 Sorry, your message to ava@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
3,"30 Sorry, your message to bilallasuru@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
4,"30 Sorry, your message to cathiegriggs278@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
5,4.2.2 The recipient's inbox is out of storage space. Please direct the\n4.2.2 recipient to\n4.2.2 https://support.google.com/mail/?p=OverQuotaTemp a8-20020a05620a102800b0077bcea051c6si935804qkk.550 - gsmtp


In [5]:
# Here I was checking that the values were correct and providing a second opinion on whether the keywords were well chosen,
# just without using .head 
no_existent_df.head()

,failure_message
1,"1 Requested mail action aborted, mailbox not found"
10,5.1.1 <7063665190@vtext.com> recipient address rejected
11,5.1.1 <cloud3@digibazar.store>: Recipient address rejected: undeliverable address: host fr-int-smtpin8.hostinger.io[private/dovecot-lmtp] said: 550 5.1.1 <cloud3@digibazar.store> User doesn't exist: cloud3@digibazar.store (in reply to RCPT TO command)
12,"5.1.1 <info777@schmerz-freiheit.ch>: Email address could not be found, or was misspelled (G8)"
13,5.1.1 <jon.thorpe@mrcopper.com> Recipient not found. <https://www.secureserver.net/help/fix-rejected-email-with-a-bounce-error-40685?pl_id=1592&prog_id=domainspricedright#irbpl_id=1592&prog_id=domainspricedright#irb>


In [ ]:
# Filter rows where the 'failure_message' column contains any of the terms in 'sure_keywords'
filtered_rows = df[df['failure_message'].str.contains('|'.join(sure_keywords), case=False, na=False)]

# Get unique email addresses
emails = filtered_rows['email'].drop_duplicates()

# Save to a CSV file
emails.to_csv('no_existent_emails.csv', index=False)
